# Statefarm Data - Phase 6 - Remove Dropout & Fine Tune ->weights5

Removal of dropout and training weights4vgg16final for 1 epoch to produce weights5vgg16final. Adam, lr=0.00001

Notes:
1. Training of the model below had 18 cases of marginal training data removed
2. Data augmentation was applied to approx. 20k training images (approx 1400 validation images held out)
3. No pseudo labeled images were used during training.


In [1]:
import theano
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
IMPORT_DIR = '/home/ubuntu/nbs'
%cd $IMPORT_DIR

/home/ubuntu/nbs


In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import daveutils
from daveutils import *
import davenet
from davenet import *
import my_cv_modeler
from my_cv_modeler import *

Using Theano backend.


In [4]:
ALL_DATA_DIR = '/home/ubuntu/'
DATA_HOME_DIR = ALL_DATA_DIR+'statefarm/'
TRAIN_DIR = DATA_HOME_DIR+'train/'
VALID_DIR = DATA_HOME_DIR+'valid/'
SAMPLE_DIR = DATA_HOME_DIR+'sample/'
MODELS_DIR = DATA_HOME_DIR+'models/'
RESULTS_DIR = DATA_HOME_DIR+'results/'
TEST_DIR = DATA_HOME_DIR+'test/'
CACHE_DIR = DATA_HOME_DIR+'cache/'

# 1. Prepare Data

#### Identify and remove poor quality training data

Previously Identified Data that is badly classified or multi-class:

In [5]:
%cd $DATA_HOME_DIR

/home/ubuntu/statefarm


# 2. Reload our previous best Sequential Vgg16 Model 

In [6]:
#del vgg; 
vgg = Dave16()
model = vgg.model

In [7]:
#del model; 
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]
model = Sequential(conv_layers)

In [8]:
def add_bn_layers(p, model):
    new_model = model
    new_model.add(MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]))
    new_model.add(Flatten())
    new_model.add(Dropout(p/2))
    new_model.add(Dense(128, activation='relu'))
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[3].get_weights())
    new_model.add(BatchNormalization())
    new_model.add(Dropout(p/2))
    new_model.add(Dense(128, activation='relu'))
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[6].get_weights())
    new_model.add(BatchNormalization())
    new_model.add(Dropout(p))
    new_model.add(Dense(10, activation='softmax'))  
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[9].get_weights())
    return new_model

In [9]:
model = add_bn_layers(0, model)

In [10]:
#model = read_model(4, cross='vgg16final') # Error - vgg_mean is not global
model.load_weights(os.path.join(CACHE_DIR, 'model_weights4vgg16final.h5'))

In [11]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]            
___________________________________________________________________________________________

zeropadding2d_13 (ZeroPadding2D) (None, 512, 16, 16)   0           convolution2d_12[0][0]           
                                                                   convolution2d_12[1][0]           
____________________________________________________________________________________________________
convolution2d_13 (Convolution2D) (None, 512, 14, 14)   2359808     zeropadding2d_13[0][0]           
                                                                   zeropadding2d_13[1][0]           
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 512, 7, 7)     0           convolution2d_13[1][0]           
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 25088)         0           maxpooling2d_6[0][0]             
___________________________________________________________________________________________

In [12]:
for i, layer in enumerate(model.layers):
    #if "batch" in layer.name:
        #new_model.add(BatchNormalization())
    #else:
        #new_model.add(layer)
    if "dropout" in layer.name:
        layer.set_weights([o*0.5 for o in layer.get_weights()])
        print("layer#",i, layer.name, "weights halved")  


layer# 33 dropout_3 weights halved
layer# 36 dropout_4 weights halved
layer# 39 dropout_5 weights halved


In [13]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 3. Train the Model - without dropout

n.b. Mixiterator was not used.  Only test data having a prediction probability >0.995 has been used.
This data is considered to be of such good quality that it can be mixed with real data. The pseudo training data will make up 43% of the training data at this stage (39% after validation data is added). Yes, it's a little high, but lets see how it goes.. 

In [14]:
%cd $ALL_DATA_DIR'statefarm1/'

/home/ubuntu/statefarm1


In [15]:
gen = ImageDataGenerator()
val_generator = gen.flow_from_directory(
        'valid',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)
val_generator.N

Found 1410 images belonging to 10 classes.


1410

In [16]:
dgen = ImageDataGenerator(  rotation_range=3,
                            width_shift_range=0.1,
                            height_shift_range=0.05,
                         )
tgenerator = dgen.flow_from_directory(
        'train',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True
)

Found 20996 images belonging to 10 classes.


In [17]:
model.optimizer.lr=0.00001
model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N
)

Epoch 1/1
20996/20996 [==============================] - 1591s - loss: 0.0317 - acc: 0.9910 - val_loss: 0.0204 - val_acc: 0.9936


In [18]:
save_model(model, 5, cross='vgg16final')

# Submit Results

In [19]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [26]:
test_generator = gen.flow_from_directory(
        'test',
        target_size=(224, 224),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)
test_generator.N

Found 79726 images belonging to 1 classes.


79726

In [27]:
preds = model.predict_generator(test_generator, test_generator.N)
#generator, val_samples, max_q_size=10, nb_worker=1

In [28]:
subm = do_clip(preds,0.93)

In [29]:
subm_name = os.path.join(ALL_DATA_DIR,'statefarm1','results','subm1.gz')

In [30]:
batches = get_batches(os.path.join(ALL_DATA_DIR,'statefarm1','train'), batch_size=65)

Found 20996 images belonging to 10 classes.


In [32]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(os.path.join(ALL_DATA_DIR,'statefarm1'))

Found 20996 images belonging to 10 classes.
Found 1410 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [33]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [35]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.927853,0.046280,0.007778,0.020082
1,img_14887.jpg,0.025973,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007894,0.930000
2,img_62885.jpg,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778
3,img_45125.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.103542,0.007778,0.887107,0.007778
4,img_22633.jpg,0.093537,0.007778,0.007778,0.007778,0.007778,0.711316,0.007778,0.007778,0.161852,0.028624


In [36]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [37]:
FileLink(subm_name)

/home/ubuntu/statefarm1/results/subm1.gz

# Conclusion

This model achieved a loss of approx 0.69 and place of around 30% against the leader board.  Clearly further work needs to be done.